In [ ]:
# Downloading the required packages
!pip install pymsteams
!pip install google_spreadsheet
!pip install google-auth-oauthlib

In [ ]:
# Use this only if you are running this code on Google Colab
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Importing the required packages
import requests
import pandas as pd
import urllib 
from urllib.request import urlopen
import ssl
import json
from pandas.io.json import json_normalize

# SSL verification
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
# First way to download the data
url = "https://data.epa.ie/bw/api/v1/Alerts"  # I am using a public open source URL for this project
response = requests.get(url)
data = response.json()

In [ ]:
# An Alternative solution to download the data
# Create a function to get data from the API link
def getjson(sourceurl):
  html = urlopen(sourceurl)
  contents = html.read()
  contents = json.loads(contents.decode('utf-8'))
  html = urlopen(sourceurl + "?page=1&per_page={per_page}".format(per_page=contents['count']))
  contents = html.read()
  contents = json.loads(contents.decode('utf-8'))
  df1 = json_normalize(contents["list"])  # Writing Json data to Pandas Dataframe
  return df1

In [ ]:
# Storing the data to a CSV File
alerts_data = getjson("https://data.epa.ie/bw/api/v1/Alerts")
alerts_data.to_csv('Path pointing the location you want to save your file to!')

In [ ]:
# For Storing data to Google sheets you would need these Credentials. Look up at the guide for steps.
scopes = ['https://spreadsheets.google.com/feeds',
          'https://www.googleapis.com/auth/drive',]

credentials = ServiceAccountCredentials.from_json_keyfile_name('File path to the api key in json!',scopes=scopes)
sheetid = 'Paste your google sheet ID here'

In [ ]:
# Details of the sheet you want to store the data to
service = discovery.build('sheets', 'v4', credentials=credentials)
range_name="Sheet1!A1:D1"
body = {
    'majorDimension' : 'ROWS',
    'values': data.values.tolist()
}

In [ ]:
# This part will keep appending new data to your Google Sheet
service.spreadsheets().values().append(
    spreadsheetId=sheetid, range=range_name, valueInputOption='USER_ENTERED',
    insertDataOption='INSERT_ROWS', body=body).execute()

In [ ]:
# You can also send Notifications for data updates to teams
myTeamsMessage = pymsteams.connectorcard("write your connector card URL here!")
myTeamsMessage.text(data.to_html())
myTeamsMessage.send()